In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import sys
sys.path.append('../')
from MiscScripts.isomerCount import isomerCountMain
warnings.filterwarnings("ignore")

## Solubility

In [2]:
solubility = pd.read_csv("../Data/Processed/0.2.0-solubility.csv") #Importing the data
solubility.dropna(subset = "InChI", inplace = True)
print(solubility.columns)
solubility = solubility[["Compound", "SMILES", "InChI", "logS", "Temperature"]] #Selecting columns

solubility["logS-dataSource"] = "AqSolDB + IUPAC SDS + DDB 2023"
inchiList = solubility["InChI"].tolist()

Index(['Compound', 'SMILES', 'Temperature', 'InChI', 'logS'], dtype='object')


In [3]:
lowe = pd.read_csv("../Data/Processed/0.0.0-LoweSol.csv") #Importing the data
lowe.dropna(subset = "InChI", inplace = True)
lowe = lowe[["Compound", "SMILES", "InChI", "logS", "Temperature"]] #Selecting columns

lowe["logS-dataSource"] = "Lowe et al, 2023"
inchiList.extend(lowe["InChI"].tolist())

## Henry's Law

In [4]:
hl = pd.read_csv("../Data/Processed/HenrysLaw/2.7.1-HenrysLaw.csv") #Importing the data
hl.dropna(subset = "InChI", inplace = True)
print(hl.columns)
hl = hl[["Compound", "SMILES", "Temperature", "InChI", "Constants"]] #Selecting columns
hl = hl.rename(columns={"Constants": "HenryConstant"}) #Renaming columns

hl["HenryConstant-dataSource"] = "https://doi.org/10.5194/acp-23-10901-2023"
inchiList.extend(hl["InChI"].tolist())

Index(['Compound', 'InChIKey', 'CAS', 'Constants', 'Temperature', 'SMILES',
       'InChI'],
      dtype='object')


## Combining the Data

In [5]:
inchiList = np.unique(inchiList)
print(len(inchiList))
dataSources = [solubility, lowe, hl]

master = pd.DataFrame()

for inchi in tqdm(inchiList):
    subList = []
    for ds in dataSources:
        subList.append(ds[ds["InChI"] == inchi])

    subConcat = pd.concat(subList)

    master = pd.concat([master, subConcat])

master.drop_duplicates(inplace = True)
print(master.shape)

df = isomerCountMain(master)
df.to_csv("../Data/Combined/0.5.0-Master.csv", index = False)

18006


100%|██████████| 18006/18006 [01:58<00:00, 152.26it/s]


(47619, 8)


Adding isomer counts:  53%|█████▎    | 25190/47619 [00:06<00:04, 5301.90it/s][14:37:22] Cannot assign bond directions!
[14:37:22] Cannot assign bond directions!
Adding isomer counts: 100%|██████████| 47619/47619 [00:09<00:00, 4806.44it/s]


[]
